In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.models.rnn import rnn, rnn_cell, seq2seq

from data_generator import get_batch
import utils

In [2]:
# vars
batch_size = 2
seq_len = 5
in_size = 15
l_f_N = 20
out_size = 10

# tf vars
inputs = tf.placeholder(tf.float32, [batch_size, seq_len, in_size])
inputs_split = tf.split(split_dim = 1, num_split = seq_len, value = inputs)
inputs_split = [tf.squeeze(input_) for input_ in inputs_split]
targets = tf.placeholder(tf.float32, [batch_size, out_size])
l_f_cell = rnn_cell.BasicRNNCell(l_f_N)
l_f_initial_state = l_f_cell.zero_state(batch_size, tf.float32)

In [3]:
# rnn shizzle
outputs, state = rnn.rnn(l_f_cell, inputs_split, initial_state=l_f_initial_state, dtype=None, sequence_length=None, scope=None)

In [4]:
output = outputs[-1]#tf.pack(outputs) to get the last layer

# l_out
with tf.variable_scope('out'):
    W = tf.Variable(tf.truncated_normal(shape=[l_f_N, out_size]), name="W")
    b = tf.Variable(tf.zeros(shape=[out_size]), name="b")

# Layer

#outputs = tf.reshape(outputs, [batch_size*seq_len, l_f_N])

l_out = tf.matmul(output, W) + b

#l_out = tf.reshape(l_out, [batch_size, seq_len, out_size])

loss = tf.nn.sigmoid_cross_entropy_with_logits(l_out, targets)
#    [tf.reshape(l_out, [-1, out_size])],
#    [tf.reshape(targets, [-1])],
#    [tf.ones([batch_size * seq_len])],
#    out_size)
train_step = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)

In [5]:
def myPad(input_vector, max_length):
    seq_len = input_vector.shape[1]
    out = np.zeros((input_vector.shape[0], max_length, input_vector.shape[2]), dtype='int32')
    out[:,:seq_len, :] = input_vector
    return out
        

In [6]:
# Train
train_t, train_X, train_mask = get_batch(batch_size=batch_size)
train_t_oh = utils.one_hot(train_t, m=10).astype('float32')
train_X_oh = myPad(utils.one_hot(train_X, m=15), 5).astype('float32')

In [7]:
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

In [9]:
for i in range(10):
    john = sess.run([train_step, loss], feed_dict={inputs: train_X_oh, targets: train_t_oh})
    print(np.mean(john[1]))

0.732392
0.699256
0.668558
0.640114
0.613754
0.58932
0.566661
0.54564
0.526127
0.508002
